# Заполнение отчета по дивидендам иностранных эмитентов на nalog.ru

## Автор

Alexander Gerasiov <a@gerasiov.net>

http://github.com/gerasiov/3ndfl

## Лицензия

GPL версии 2 или более поздняя

## Подробнее

Читайте в README.md

In [ ]:
# Install selenium into venv or user's dir

#!pip install selenium
#!pip install --user selenium

# Or install is manually to your system with
# sudo apt install python3-selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from decimal import Decimal
import getpass
import csv

In [ ]:
driver = None

def open_browser(executable):
    global driver
    options = webdriver.chrome.options.Options()

    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_argument('--incognito')
 
    prefs = {
        'safebrowsing.enabled': True,
        'credentials_enable_service': False,
        'profile.password_manager_enabled' : False,
        'credentials_enable_service': False,
        'profile.password_manager_enabled': False
    }
    options.add_experimental_option('prefs', prefs)
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-infobars')
    options.add_argument('--safebrowsing-disable-extension-blacklist')
    options.add_argument('--safebrowsing-disable-download-protection')


    service = webdriver.chrome.service.Service(executable_path=executable)
    driver = webdriver.Chrome(service=service, options=options)

In [ ]:
def login():
    login_url = 'https://lkfl2.nalog.ru/lkfl/login'
    logged_in = 'https://lkfl2.nalog.ru/lkfl/'
    driver.get(login_url)
    driver.implicitly_wait(0.5)
    if driver.current_url == logged_in:
        # Already logged in
        return
    login_link = driver.find_element(by=By.LINK_TEXT, value='Войти через госуслуги (ЕСИА)')
    login_link.click()
    driver.implicitly_wait(1)
    if driver.current_url == logged_in:
        # Already logged in
        return
    print('Enter ESIA login name:')
    username = input()
    print('Enter ESIA password:')
    password = getpass.getpass()
    
    f = driver.find_element(by=By.ID, value='login')
    f.clear()
    f.send_keys(username)
    f = driver.find_element(by=By.ID, value='password')
    f.clear()
    f.send_keys(password)
    f = driver.find_element(by=By.ID, value='loginByPwdButton')
    f.click()
    driver.implicitly_wait(1)
    
    if driver.current_url == logged_in:
        # Already logged in
        return

    try:
        f = driver.find_element(by=By.ID, value='smscode')
        print('Enter SMS OTP:')
        sms = input()
        f.clear()
        f.send_keys(sms)
        f = driver.find_element(by=By.ID, value='confirm_otp')
        f.click()
    except NoSuchElementException:
        print('No SMS request found')
        pass
    
    if driver.current_url != logged_in:
        print('Something unexpected happens, log in manually, please...')


In [ ]:
def click_next():
    p = '//button[text()="Далее"]'
    wait_for_elemet(p)
    button = driver.find_element(by=By.XPATH, value=p)
    button.click()

In [ ]:
def wait_for_elemet(path, retry=10):
    for ret in range(1, retry + 1):
        try:
            driver.find_element(by=By.XPATH, value=path)
            break
        except NoSuchElementException:
            if ret == retry:
                raise
            driver.implicitly_wait(1)

In [ ]:
foreing_income_tab_path = '//li[text()="За пределами РФ"]'

def get_foreing_income_tab():
    return driver.find_element(by=By.XPATH, value=foreing_income_tab_path).get_attribute('aria-controls')

def open_3nfdl_foreing_income_form():
    driver.get('https://lkfl2.nalog.ru/lkfl/situations/3NDFL')
    driver.implicitly_wait(0.5)

    click_next()
    wait_for_elemet(foreing_income_tab_path)
    tab = driver.find_element(by=By.XPATH, value=foreing_income_tab_path)
    tab.click()
    driver.implicitly_wait(0.5)


In [ ]:
def click_button(tab, text):
    
    wait_for_elemet(f'//div[@id="{tab}"]//button[text()="{text}"]')
    button = driver.find_element(
        by=By.XPATH, 
        value=f'//div[@id="{tab}"]//button[text()="{text}"]')
    button.click()

In [ ]:
def fill_text(tab, income_number, field, value):
    i=driver.find_element(
        by=By.XPATH, 
        value=f'//div[@id="{tab}"]//input[@name="Ndfl3Package.payload.sheetB.sources[{income_number}].{field}"]')
    i.send_keys(Keys.CONTROL,"a")  # Have to delete manually because of "smart" JS
    i.send_keys(Keys.DELETE)
    i.clear()
    i.send_keys(value)
    driver.implicitly_wait(0.5)
    
def fill_date(tab, income_number, field, value):
    i=driver.find_element(
        by=By.XPATH, 
        value=f'//div[@id="{tab}"]//div[@id="Ndfl3Package.payload.sheetB.sources[{income_number}].{field}"]//input')
    i.send_keys(Keys.CONTROL,"a")  # Have to delete manually because of "smart" JS
    i.send_keys(Keys.DELETE)
    i.clear()
    i.send_keys(value, Keys.ENTER)
    
    
def fill_dropdown(tab, income_number, field, value):
    i=driver.find_element(
        by=By.XPATH, 
        value=f'//div[@id="{tab}"]//div[@id="Ndfl3Package.payload.sheetB.sources[{income_number}].{field}"]//input')
    i.send_keys(value, Keys.ENTER)
    
def click_currency_online_checkbox(tab, income_number):
    i=driver.find_element(
        by=By.XPATH, 
        value=f'//div[@id="{tab}"]//div[contains(@class,"IncomeSources_incomeSourceSpoiler_")][{income_number+1}]//input[@name="getCurrencyOnline"]')
    i.send_keys(Keys.SPACE)
    driver.implicitly_wait(0.5)

In [ ]:
def add_income(source_name, source_country, dest_country, amount, income_date, income_type, currency, income_tax):
    foreing_income_tab = get_foreing_income_tab()
    click_button(foreing_income_tab, 'Добавить источник дохода')
    incomes = driver.find_elements(
        by=By.XPATH, 
        value=f'//div[@id="{foreing_income_tab}"]//div[starts-with(@class,"IncomeSources_title_")]')
    income_number = len(incomes) - 1
    incomes[income_number].click()
    
    fill_text(foreing_income_tab, income_number, 'incomeSourceName', source_name)
    fill_dropdown(foreing_income_tab, income_number, 'oksmIst', source_country)
    fill_dropdown(foreing_income_tab, income_number, 'oksmZach', dest_country)
    fill_text(foreing_income_tab, income_number, 'incomeAmountCurrency', amount)
    fill_date(foreing_income_tab, income_number, 'incomeDate', income_date)
    fill_dropdown(foreing_income_tab, income_number, 'incomeTypeCode', income_type)
    if income_tax:
        fill_date(foreing_income_tab, income_number, 'taxPaymentDate', income_date)
        fill_text(foreing_income_tab, income_number, 'paymentAmountCurrency',income_tax) 
    fill_dropdown(foreing_income_tab, income_number, 'taxDeductionCode', 'Не предоставлять')
    fill_dropdown(foreing_income_tab, income_number, 'currencyCode', currency)

    click_currency_online_checkbox(foreing_income_tab, income_number)

In [ ]:
# Mappings from country/currency name abbrv. to OKSM code

def get_country_code(country):
    return {
        'Россия': 643,
        'РФ': 643,
        'США': 840,
        'Германия': 276,
        'ДЖЕРСИ': 832,
        'Нидерланды': 528,
    }[country]

def get_currency_code(currency):
    return {
        'RUB': 643,
        'USD': 840,
        'EUR': 978
    }[currency]

In [ ]:
# Should return list of dicts e.g
# [
#     {
#         'date': '01.01.2021',
#         'name': 'Yandex B.V.',
#         'country': 'Нидерланды',
#         'currency': 'USD',
#         'amount': '20.20',
#         'tax': '0.20'
#     },
# ]
def read_report(filename):
    result = []
    with open(filename, 'r') as file:        
        fields = ['date', 'country', 'currency', 'amount', 'tax', 'name']
        tsv_file = csv.DictReader(file, fieldnames=fields, delimiter="\t")
        for line in tsv_file:
            result.append(line)
    return result
        


In [ ]:
# Pass full name of driver's executable
# Note, that in Debian/Ubuntu you should install it with chromium-chromedriver package
open_browser(executable='/usr/bin/chromedriver')

In [ ]:
login()

In [ ]:
open_3nfdl_foreing_income_form()

In [ ]:
for record in read_report('dividends.tsv'):
    print(record)
    add_income(
        source_name=record['name'],
        source_country=get_country_code(record['country']),
        dest_country=get_country_code('РФ'),
        amount=record['amount'],
        income_date=record['date'],
        income_type='1010',  #dividend
        currency=get_currency_code(record['currency']),
        income_tax=record['tax']
    )


In [ ]:
# This one may fail if you have not filled some required fields on other tabs
click_next()